

### Generación de Modelos Predictivos

Este notebook se encarga de generar modelos predictivos.  Se puede correr en Google Colab o en el iMac.

Para usarlo en Google Colab, usar tipo_a_usar='GPU'
Para usarlo en el imac, usar tipo_a_usar='CPU'



In [1]:
tipo_a_usar='CPU'
import pandas as pd

Los datos de origen es el mismo sea desde Google Colab o desde el iMac.  La diferencia es que el primero usar referencia a Google Drive y el otro acceso directo desde el sistema operativo

In [2]:

dfbase = pd.read_pickle('checkpoints/DATOS_LIMPIOS.pkl')


In [3]:
df = dfbase.copy()
df = df[df['LABEL']!=-1]

X = df.drop('LABEL', axis=1)  # Reemplaza 'target_column' con el nombre de tu columna objetivo
y = df['LABEL']  # Reemplaza 'target_column' con el nombre de tu columna objetivo

In [30]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer
from sklearn.metrics import roc_auc_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow import keras
from tensorflow.keras import layers

In [46]:


# Cargar tus datos en X (características) y y (etiquetas)
# Asumiendo que ya tienes tus datos cargados, siéntete libre de reemplazar esta parte

# Dividir los datos en conjuntos de entrenamiento (train) y prueba (test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# BALANCAMOS X_train y y_train
#!pip install imbalanced-learn
# from imblearn.over_sampling import SMOTE
# smote = SMOTE(sampling_strategy='auto')
# X_train, y_train = smote.fit_resample(X_train, y_train)

# Normalizar los datos utilizando StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



In [47]:
from keras.models import Sequential
from keras.layers import Dense

def create_model(num_hidden_layers=1, num_neurons=128, learning_rate=0.01):
    model = Sequential()
    model.add(Dense(num_neurons, input_dim=1953, activation='relu'))
    
    t_num_neurons = num_neurons
    for _ in range(num_hidden_layers - 1):
        model.add(Dense(t_num_neurons, activation='relu'))
        t_num_neurons = t_num_neurons*(0,75)  # se reduce la cantidad de neuronas al 75%
    
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=learning_rate), metrics=[tf.keras.metrics.AUC()])
    # Compilar el modelo


    
    return model

In [48]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'num_hidden_layers': [1, 2, 3, 4],
    'num_neurons': [1000, 750, 500, 250],
    'learning_rate': [0.1, 0.01, 0.001]
}

model = KerasClassifier(build_fn=create_model, verbose=0)
roc_auc_scorer = make_scorer(roc_auc_score, greater_is_better=True)

# 7. Búsqueda en cuadrícula con GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=roc_auc_scorer, cv=3)
grid_result = grid.fit(X_train, y_train)


/var/folders/tx/1tx9w9fj6v78n5f12zkwg_vc0000gn/T/ipykernel_37477/519310345.py:9: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, verbose=0)


56/56 [==============================] - 0s 888us/step


/Users/jaimereinoso/DESARROLLO/MCD/proyectodegrado2/venv/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
72 fits failed out of a total of 144.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
72 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/jaimereinoso/DESARROLLO/MCD/proyectodegrado2/venv/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/jaimereinoso/DESARROLLO/MCD/proyectodegrado2/venv/lib/python3.11/site-packages/keras/wrappers/scikit_learn.py", line 248, in fit
    return super().fit(x, y, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
 

In [49]:
grid_result = grid.fit(X_train, y_train)


56/56 [==============================] - 0s 964us/step


/Users/jaimereinoso/DESARROLLO/MCD/proyectodegrado2/venv/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
72 fits failed out of a total of 144.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
72 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/jaimereinoso/DESARROLLO/MCD/proyectodegrado2/venv/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/jaimereinoso/DESARROLLO/MCD/proyectodegrado2/venv/lib/python3.11/site-packages/keras/wrappers/scikit_learn.py", line 248, in fit
    return super().fit(x, y, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
 

In [50]:
best_params = grid_result.best_params_
best_roc_auc = grid_result.best_score_

print("Mejores parámetros:", best_params)
print("Mejor precisión:", best_roc_auc)


Mejores parámetros: {'learning_rate': 0.001, 'num_hidden_layers': 1, 'num_neurons': 1000}
Mejor precisión: 0.7459403106240732


In [42]:
best_model = grid_result.best_estimator_
y_pred = best_model.predict(X_test)
test_roc_auc = roc_auc_score(y_test, y_pred)

print("ROC-AUC en datos de prueba:", test_roc_auc)

42/42 [==============================] - 0s 1ms/step
ROC-AUC en datos de prueba: 0.8248430244443795


In [51]:
results_df = pd.DataFrame(grid_result.cv_results_)

In [52]:
results_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_num_hidden_layers,param_num_neurons,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,2.395844,0.225792,0.206865,0.006411,0.1,1,1000,"{'learning_rate': 0.1, 'num_hidden_layers': 1,...",0.553589,0.546503,0.540147,0.546746,0.005490,20
1,1.894684,0.021302,0.177355,0.006176,0.1,1,750,"{'learning_rate': 0.1, 'num_hidden_layers': 1,...",0.608663,0.556257,0.684883,0.616601,0.052810,14
2,1.445404,0.030151,0.166248,0.001392,0.1,1,500,"{'learning_rate': 0.1, 'num_hidden_layers': 1,...",0.617607,0.581705,0.541342,0.580218,0.031153,17
3,1.077677,0.005518,0.127254,0.007619,0.1,1,250,"{'learning_rate': 0.1, 'num_hidden_layers': 1,...",0.542112,0.579295,0.555171,0.558859,0.015402,19
4,2.885641,0.090215,0.239352,0.018789,0.1,2,1000,"{'learning_rate': 0.1, 'num_hidden_layers': 2,...",0.511214,0.530141,0.502749,0.514701,0.011451,21


In [53]:
results_df.to_excel('NN_resultados_no_balanceado.xlsx')